In [4]:
IDENTIFIER = "haydn-op1"
NUM = 3

picklePath = f"../Pickles/{IDENTIFIER}/{IDENTIFIER}_"
numPath = f"../Pickles/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"
dataPath = f"../Data/{IDENTIFIER}/{NUM}/{IDENTIFIER}_{NUM}_"

import pandas as pd
import json
import networkx as nx
import numpy as np
import itertools as iter
import matplotlib.pyplot as plt
from dwave.system import DWaveSampler, FixedEmbeddingComposite

from qubo import createBQM
from samples import duplicates, overlaps
from plots import plotLagrange


phrases = pd.read_csv(picklePath + "phrases.csv", index_col=[0,1])
G = nx.read_graphml(picklePath + "graph.graphml")
instruments = json.load(open(numPath + "instruments.json"))
embedding = json.load(open(numPath + "embedding.json", "r"))

sampler = FixedEmbeddingComposite(DWaveSampler(), embedding)

In [7]:
# Node constraint

multipliers = iter.product(np.arange(1,9,1), np.arange(1,9,1))
reads = 500

file = pd.DataFrame()

for n, e in multipliers:
    bqm = createBQM(G, phrases, instruments, n, e)
    sampleset = sampler.sample(bqm, num_reads=reads, label=f"{IDENTIFIER}_{NUM}_{reads}")
    sample = sampleset.first

    new_row = pd.DataFrame({
        "Node multiplier": [n],
        "Edge multiplier": [e],
        "Lowest energy": [bqm.energy(sample.sample)],
        "Chain break fraction": [sample.chain_break_fraction],
        "Duplicates": [duplicates(sample.sample, G)],
        "Overlaps": [overlaps(sample.sample, G)]
        })
    
    file = pd.concat([file, new_row])
    
    print(f"Recorded node multiplier {n} and edge multiplier {e}!")

file.to_csv(dataPath+"lagrange.csv", index=False, mode="a", header=False)

Recorded node multiplier 1 and edge multiplier 1!
Recorded node multiplier 1 and edge multiplier 2!
Recorded node multiplier 1 and edge multiplier 3!
Recorded node multiplier 1 and edge multiplier 4!
Recorded node multiplier 1 and edge multiplier 5!
Recorded node multiplier 1 and edge multiplier 6!
Recorded node multiplier 1 and edge multiplier 7!
Recorded node multiplier 1 and edge multiplier 8!
Recorded node multiplier 2 and edge multiplier 1!
Recorded node multiplier 2 and edge multiplier 2!
Recorded node multiplier 2 and edge multiplier 3!
Recorded node multiplier 2 and edge multiplier 4!
Recorded node multiplier 2 and edge multiplier 5!
Recorded node multiplier 2 and edge multiplier 6!
Recorded node multiplier 2 and edge multiplier 7!
Recorded node multiplier 2 and edge multiplier 8!
Recorded node multiplier 3 and edge multiplier 1!
Recorded node multiplier 3 and edge multiplier 2!
Recorded node multiplier 3 and edge multiplier 3!
Recorded node multiplier 3 and edge multiplier 4!


`haydn-op1`: VM = 6, EM = 6

`beethoven-op67`: VM = 6, EM = 12